# 딥러닝 기반 상품 카테고리 자동 분류 서버 예

### 파일에서 학습 데이터를 읽는다.

In [1]:
import os
import sys
import json
import gensim
import requests

import numpy
from numpy import array, argmax
from scipy import sparse

from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

from konlpy.tag import Kkma

import keras
import keras.preprocessing.text
from keras.utils import to_categorical

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

Using TensorFlow backend.


In [2]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])

In [3]:
#joblib.dump(y_name_id_dict,"y_name_id_dict.dat")

### text 형식으로 되어 있는 카테고리 명을 숫자 id 형태로 변환한다.

In [4]:
y_name_id_dict = joblib.load("y_name_id_dict.dat")

In [5]:
print(y_name_id_dict)

{'가구/인테리어': 11, '반려동물': 4, '도서/문구': 8, '의류': 2, '여행/e쿠폰': 15, '식품': 6, '건강': 7, '뷰티': 0, '디지털': 10, '출산/육아': 16, '스포츠/레저': 9, '잡화': 14, '컴퓨터': 3, '자동차/공구': 1, '생필품/주방': 13, '가전': 12, '취미': 5}


In [6]:
#y_name_set = set(y_text_list)
#y_name_id_dict = dict(zip(y_name_set, range(len(y_name_set))))
#print(y_name_id_dict.items())
#y_id_name_dict = dict(zip(range(len(y_name_set)),y_name_set))
y_list = [y_name_id_dict[x] for x in y_text_list]

### train test 분리하는 방법 

In [7]:
x_train, x_test , y_train, y_test = train_test_split(x_text_list, y_list, test_size=0.2, random_state=42)

## 딥러닝 기반 text 분류에 필요한 모듈 로드

#### 모델 파일을 만약 만들었다면, 아래와 같이 로드 가능하다.

In [8]:
word2vec = gensim.models.word2vec.Word2Vec.load('word_models.model')
word2vec.init_sims(replace=True)

#model = KeyedVectors.load('ko.bin')

#word2vec = model
#word2vec.init_sims(replace=True)

## Train, Test 데이터 새로 만들기

In [9]:
kkma = Kkma()

kkma_x_train = list(map(lambda x: kkma.sentences(x[1])[0], x_train))
kkma_x_test = list(map(lambda x: kkma.sentences(x[1].replace('/', '').replace('(', '') 
                .replace(')', '').replace('[', '').replace(']', ''))[0], x_test))

### text 데이터를 word-id 형태로 변환한다.

In [10]:
sequence_tokenizer = keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
sequence_tokenizer.fit_on_texts(kkma_x_train)
max_features = len(sequence_tokenizer.word_index)


def texts_to_sequences2(d_list, tokenizer, maxlen=300):
    seq = tokenizer.texts_to_sequences(d_list)
    print('mean:', numpy.mean([len(x) for x in seq]))
    print('std:', numpy.std([len(x) for x in seq]))
    print('median:', numpy.median([len(x) for x in seq]))
    print('max:', numpy.max([len(x) for x in seq]))
    seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
    return seq

In [11]:
train = texts_to_sequences2(kkma_x_train, sequence_tokenizer)
test = texts_to_sequences2(kkma_x_test, sequence_tokenizer)

mean: 12.2513235294
std: 5.73576868135
median: 11.0
max: 43
mean: 10.01
std: 5.2020827277
median: 9.0
max: 40


#### word의 embedding 형태의 weight 를 초기화 한다. 

In [12]:
input_dim = train.shape[1]

input_tensor = keras.layers.Input(shape=(input_dim,), dtype='int32')

In [13]:
word_vec_dim = 100
#word_vec_dim = 200
not_ct = 0
weights = numpy.zeros((max_features + 1, word_vec_dim))
for word, index in sequence_tokenizer.word_index.items():
    if False:
        pass
    if word in word2vec.wv.vocab:
        weights[index, :] = word2vec[word]
    else:
        not_ct+=1
        weights[index, :] = numpy.random.uniform(-0.25, 0.25, word_vec_dim)
# del word2vec
# del sequence_tokenizer
print (not_ct)

13914


####  학습할 레이어를 구성한다.

In [14]:
embedded = keras.layers.Embedding(input_dim=max_features + 1,
                                  output_dim=word_vec_dim, input_length=input_dim,
                                  weights=[weights],trainable=True)(input_tensor)

In [15]:
tensors = []
for filter_length in [3, 5]:
    tensor = keras.layers.Convolution1D(nb_filter=50, filter_length=filter_length)(embedded)
    tensor = keras.layers.Dropout(0.5)(tensor)
    tensor = keras.layers.Activation('elu')(tensor)
    tensor = keras.layers.MaxPooling1D(pool_length=input_dim - filter_length + 1)(tensor)
    tensor = keras.layers.Flatten()(tensor)
    tensors.append(tensor)

/home/dzzp/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=3)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzzp/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=298)`
  
/home/dzzp/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=5)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzzp/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=296)`
  


In [16]:
# embedded = keras.layers.Dropout(0.5)(embedded)
output_tensor = keras.layers.merge(tensors, mode='concat', concat_axis=1)
output_tensor = keras.layers.Dropout(0.5)(output_tensor) 
output_tensor = keras.layers.Dense(len(set(y_list)), activation='softmax')(output_tensor)

# output = Dense(NUM_CLASSES, input_dim = hidden_dim_2, activation = "softmax")(pool_rnn) # See equations (6) and (7).

cnn = keras.models.Model(input_tensor, output_tensor)
cnn.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
print(cnn.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 300)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 300, 100)      1882300     input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 298, 50)       15050       embedding_1[0][0]                
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 296, 50)       25050       embedding_1[0][0]                
___________________________________________________________________________________________

/home/dzzp/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [17]:
'''
cnn.fit(train, numpy.asarray(to_categorical(y_train)), batch_size=30, nb_epoch=20,
        validation_data=(test, numpy.asarray(to_categorical(y_test))))
'''
cnn.fit(train, numpy.asarray(to_categorical(y_train)), batch_size=30, epochs=20,
        validation_data=(test, numpy.asarray(to_categorical(y_test))))

Train on 6800 samples, validate on 1700 samples
Epoch 1/20
6800/6800 [==============================] - 2s - loss: 2.7706 - acc: 0.1293 - val_loss: 2.6565 - val_acc: 0.3565
Epoch 2/20
6800/6800 [==============================] - 1s - loss: 2.4526 - acc: 0.2674 - val_loss: 2.4632 - val_acc: 0.4624
Epoch 3/20
6800/6800 [==============================] - 1s - loss: 2.2072 - acc: 0.3465 - val_loss: 2.2899 - val_acc: 0.5047
Epoch 4/20
6800/6800 [==============================] - 1s - loss: 2.0094 - acc: 0.4016 - val_loss: 2.1415 - val_acc: 0.5341
Epoch 5/20
6800/6800 [==============================] - 1s - loss: 1.8762 - acc: 0.4476 - val_loss: 2.0283 - val_acc: 0.5594
Epoch 6/20
6800/6800 [==============================] - 1s - loss: 1.7541 - acc: 0.4844 - val_loss: 1.9340 - val_acc: 0.5659
Epoch 7/20
6800/6800 [==============================] - 1s - loss: 1.6625 - acc: 0.5063 - val_loss: 1.8562 - val_acc: 0.5835
Epoch 8/20
6800/6800 [==============================] - 1s - loss: 1.6013 - a

## 제출

In [18]:
mode = 'test'
#mode = 'eval'

eval_x_text_list = []
if mode == 'test':
    with open("soma8_test_data.dat",encoding=enc) as fin:
        for line in fin.readlines():
            info = json.loads(line.strip())
            eval_x_text_list.append((info['pid'],info['name']))
else:
    with open("soma8_eval_data.dat",encoding=enc) as fin:
        for line in fin.readlines():
            info = json.loads(line.strip())
            eval_x_text_list.append((info['pid'],info['name']))

kkma_eval_x_text_list = list(map(lambda x: kkma.sentences(x[1].replace('/', '').replace('(', '')
                            .replace(')', '').replace('[', '').replace(']', ''))[0], eval_x_text_list))

In [19]:
#eval_x_list = texts_to_sequences2(map(lambda i : i[1],eval_x_text_list),sequence_tokenizer)
#eval_x_text_list = texts_to_sequences2(kkma_eval_x_text_list, sequence_tokenizer)
#eval_x_list = texts_to_sequences2(map(lambda i : i[1],eval_x_text_list),sequence_tokenizer)
eval_x_list = texts_to_sequences2(kkma_eval_x_text_list, sequence_tokenizer)

mean: 10.0835913313
std: 4.96387664158
median: 9.0
max: 39


In [20]:
#pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))
#pred = cnn.predict(eval_x_list)
pred = cnn.predict(eval_x_list)
pred_list = [argmax(y) for y in pred]

In [21]:
name='하지윤'
nickname='punk_zzang3'

param = {
    'pred_list': ",".join(map(lambda i : str(int(i)), pred_list)),
    'name': name,
    'nickname': nickname,
    'mode': mode
}
d = requests.post('http://eval.buzzni.net:20001/eval', data=param)
print(d.json())

{'msg': 'If you pull docker image before 2017-09-27 21:30,  pull your docker image again.', 'precision': 0.6896284829721362}
